In [335]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bliss-kaggle-challenge/example_submission.csv
/kaggle/input/bliss-kaggle-challenge/train.csv
/kaggle/input/bliss-kaggle-challenge/test.csv


In [336]:
df_train = pd.read_csv('/kaggle/input/bliss-kaggle-challenge/train.csv')
df_test = pd.read_csv('/kaggle/input/bliss-kaggle-challenge/test.csv')
df_example = pd.read_csv('/kaggle/input/bliss-kaggle-challenge/example_submission.csv')

In [337]:
train_data = df_train.copy()
test_data = df_test.copy()

In [338]:
contained = 0

for names in test_data:
    for names_train in train_data['Name']:
        if names == names_train:
            contained += 1
            
print(contained)            

0


In [339]:
def split_names(full_name):
    return full_name.split(" ")[-1]
split_names(str1)

'Baketton'

In [340]:
#data

In [341]:
#data['HomePlanet'].value_counts()

In [342]:
#data.isna().sum()/len(data)

In [343]:
#df_test = df['Name']
#df_test.map(split_names, na_action='ignore')
#len(df_test)

In [344]:
train_data = train_data.dropna()
#test_data = test_data.dropna()

In [345]:
from sklearn.preprocessing import OrdinalEncoder

In [346]:
#data

In [347]:
test_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews
1,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston
2,0009_01,Mars,False,F/1/P,TRAPPIST-1e,32.0,False,73.0,0.0,1123.0,0.0,113.0,Berers Barne
3,0014_01,Mars,False,F/3/P,55 Cancri e,27.0,False,1286.0,122.0,NaN,0.0,0.0,Flats Eccle
4,0020_01,Earth,True,E/0/S,TRAPPIST-1e,1.0,False,0.0,0.0,0.0,0.0,0.0,Almary Brantuarez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9268_01,Earth,True,G/1505/P,TRAPPIST-1e,31.0,False,0.0,0.0,0.0,0.0,0.0,Agnesa Baldson
1996,9270_01,Earth,True,G/1497/S,55 Cancri e,33.0,False,0.0,0.0,0.0,0.0,0.0,Lan Mckinsond
1997,9272_02,Earth,False,F/1894/P,TRAPPIST-1e,21.0,False,86.0,3.0,149.0,208.0,329.0,Gordo Simson
1998,9274_01,NaN,True,G/1508/P,TRAPPIST-1e,23.0,False,0.0,0.0,0.0,0.0,0.0,Chelsa Bullisey


In [348]:
train_data['Name'] = train_data['Name'].apply(split_names)
test_data['Name'] = test_data['Name'].fillna(" ").apply(split_names)

In [349]:
#data

In [350]:
#test = data['HomePlanet']

def create_lookup_table(data):
    arr_size = len(data.unique())
    my_list = []
    my_dict = {}
    unique_data = data.unique().tolist()
    for d in range(arr_size):
        my_dict[unique_data[d]] = d+1
    return my_dict

def HotEncode(x):
    lookup_table = create_lookup_table(x)
    func_dict = lambda y: lookup_table[y]
    return x.apply(func_dict)

non_numerical_columns = ['HomePlanet', 'Cabin', 'Destination', 'Name']

def OneHotEncode(dataframe, to_list):
    lookup_table_list = []
    for name in to_list:
        lookup_table = create_lookup_table(dataframe[name])
        lookup_table_list.append(lookup_table)
        dataframe[name] = HotEncode(dataframe[name])
    
    return dataframe, lookup_table_list

train_data, look_up_table_data = OneHotEncode(train_data, non_numerical_columns)
test_data, _ = OneHotEncode(test_data, non_numerical_columns)

train_data['VIP'] = train_data['VIP'].replace({True: 1, False: 0}) ## From Bool to Int
train_data['Transported'] = train_data['Transported'].replace({True: 1, False: 0}) ## From Bool to Int
train_data['CryoSleep'] = train_data['CryoSleep'].replace({True: 1, False: 0}) ## From Bool to Int

test_data['VIP'] = test_data['VIP'].replace({True: 1, False: 0}) ## From Bool to Int
#test_data['Transported'] = test_data['Transported'].replace({True: 1, False: 0}) ## From Bool to Int
test_data['CryoSleep'] = test_data['CryoSleep'].replace({True: 1, False: 0}) ## From Bool to Int

train_data = train_data.drop(columns = ['PassengerId'])
test_data = test_data.drop(columns = ['PassengerId'])

In [351]:

#data

In [352]:
X_train = train_data.drop(columns = ['Transported'])
y_train = train_data['Transported']

In [353]:
X_test = test_data

In [354]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                 max_depth=1, random_state=0).fit(X_train, y_train)


In [355]:
X_test = X_test.fillna(0)

In [356]:
clf.predict(X_test)

array([0, 1, 1, ..., 1, 1, 0])

In [357]:
df_example['Transported'] = clf.predict(X_test)
df_example

,PassengerId,Transported
0,0005_01,0
1,0007_01,1
2,0009_01,1
3,0014_01,0
4,0020_01,1
...,...,...
1995,9268_01,1
1996,9270_01,1
1997,9272_02,1
1998,9274_01,1


In [358]:
df_example.to_csv('submission.csv', index = False)

In [359]:
df_example = df_example.replace({0: False, 1: True}) ## From Bool to Int

In [360]:
df_example.to_csv('submission.csv', index = False)

In [361]:
df_example

,PassengerId,Transported
0,0005_01,False
1,0007_01,True
2,0009_01,True
3,0014_01,False
4,0020_01,True
...,...,...
1995,9268_01,True
1996,9270_01,True
1997,9272_02,True
1998,9274_01,True
